In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mnist_example import MNISTModel, MNISTDataModule

datamodule = MNISTDataModule(batch_size=10)
datamodule.setup()

len(datamodule.test_dataloader())

10

In [3]:
from energizer.data.datamodule import ActiveDataModule

dm = ActiveDataModule(datamodule=datamodule)
dm.setup()
dm.setup_folds()

dm.is_on_pool = False
print(len(dm.test_dataloader()))
dm.is_on_pool = True
print(len(dm.test_dataloader()))

10
10


In [4]:
# print(
#     len(dm.pool_fold), 
#     len(dm.test_dataloader()),
#     len(dm.train_fold), 
#     len(dm.train_dataloader()),
# )
# dm.label(list(range(100)))
# print(
#     len(dm.pool_fold), 
#     len(dm.test_dataloader()),
#     len(dm.train_fold), 
#     len(dm.train_dataloader()),
# )

---

In [5]:
import torch
from torch.utils.data import DataLoader
from energizer.learners.base import Deterministic
from energizer.learners.acquisition_functions import entropy
from pytorch_lightning import LightningModule, Trainer as pl_Trainer
from energizer.trainer import Trainer

In [6]:
class ActiveLearner(Deterministic):

    def pool_step(self, batch, *args, **kwargs) -> torch.Tensor:
        x, _ = batch
        logits = self(x)
        scores = entropy(logits)
        return scores

In [ ]:
is

In [7]:
model = MNISTModel()
active_learner = ActiveLearner(model)

In [8]:
pl_trainer = pl_Trainer(max_epochs=1)
pl_trainer.fit(active_learner, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | learner | MNISTModel | 184 K 
---------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:589: UserWarning: You have overridden `on_before_batch_transfer` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.
  f"You have overridden `{hook_name}` in `LightningModule` but have passed in a"
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:589: UserWarning: Yo

Epoch 0: 100%|██████████| 20/20 [00:00<00:00, 67.76it/s, loss=2.45, v_num=316]


In [9]:
pl_trainer.test(active_learner, datamodule)

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 221.67it/s]


[{}]

In [61]:
class A:
    pass

class B(A):
    pass

class C(B):
    pass

In [63]:
isinstance(C(), A)

True

In [60]:
from energizer.learners.base import Learner
isinstance(active_learner, Learner)

False

In [56]:
trainer = Trainer(query_size=2, max_epochs=1)
trainer.active_fit(active_learner, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | learner | MNISTModel | 184 K 
---------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


DIO                                                                         
current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf, 2.1421, 2.2434, 2.2774, 2.2500, 2.2899, 2.2540, 2.2548,
        2.2771, 2.2657, 2.2670], grad_fn=<CatBackward0>)
all_indices: tensor([-1, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
top_scores: tensor([2.2899, 2.2774], grad_fn=<TopkBackward0>)
top_indices: tensor([4, 2]) 

False


ValueError: Expected a parent

In [52]:
from pytorch_lightning.utilities.model_helpers import is_overridden

In [54]:
is_overridden("pool_epoch_end", active_learner)

ValueError: The parent should define the method

In [ ]:
trainer.test(model, datamodule)

In [ ]:
test_dl = DataLoader(
    [datamodule.mnist_test[i] for i in range(10)], 
    shuffle=False, 
    batch_size=2,
)
x, y = next(iter(test_dl))

In [ ]:
model = MNISTModel()
adapter = MCDropout(model)
strategy = EntropyStrategy(adapter, 3)
dm = ActiveDataModule(datamodule=datamodule)
dm.setup()
dm.setup_folds()

In [ ]:
len(dm.pool_fold), len(dm.train_fold)

In [ ]:
datamodule.train_dataloader().batch_size

In [ ]:
trainer = Trainer(max_epochs=1, enable_progress_bar=True, total_budget=3)
trainer.active_fit(strategy, datamodule=dm)

In [ ]:
len(dm.val_dataloader())

In [ ]:
trainer.fit_loop.done

In [ ]:
trainer.fit(strategy, datamodule=datamodule)

In [ ]:
trainer.current_epoch